In [ ]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta

# Load the Transaction table
transaction_df = pd.read_csv('/mnt/data/13.Transaction.csv')

# Function to generate random dates within a range
def random_date(start, end):
    return start + timedelta(days=random.randint(0, int((end - start).days)))

# Generating sample data for the Contract table
contract_data = {
    "ContractID": [f"C{i:04d}" for i in range(1, len(transaction_df) + 1)],
    "TransactionID": transaction_df["TransactionID"],
    "ClientID": transaction_df["ClientID"],
    "PropertyID": transaction_df["PropertyID"],
    "StartDate": [],
    "EndDate": [],
    "ContractType": transaction_df["TransactionType"],
    "Amount": transaction_df["PurchasePrice"].fillna(transaction_df["LeasePricePerYear"] * transaction_df["LeaseYear"]),
    "ContractStatus": [],
    "RenewalOption": [random.choice(["Yes", "No"]) for _ in range(len(transaction_df))],
    "DepositAmount": [round(random.uniform(1000, 5000), 2) for _ in range(len(transaction_df))],
    "MonthlyRent": round(transaction_df["LeasePricePerYear"] / 12, 2).fillna(0),
    "TerminationClause": [random.choice(["No penalty", "1 month notice required", "2 months notice required"]) for _ in range(len(transaction_df))],
    "SignedDate": [],
    "WitnessName": [random.choice(["John Doe", "Jane Smith", "Alice Johnson", "Bob Brown"]) for _ in range(len(transaction_df))],
    "SpecialTerms": [random.choice(["", "Pet allowed", "No smoking", "Maintenance included"]) for _ in range(len(transaction_df))]
}

# Populate the StartDate, EndDate, SignedDate, and ContractStatus
for i, row in transaction_df.iterrows():
    transaction_date = datetime.strptime(row["TransactionDate"], '%Y-%m-%d')
    start_date = random_date(transaction_date + timedelta(days=1), transaction_date + timedelta(days=365))
    signed_date = transaction_date
    contract_data["StartDate"].append(start_date.strftime('%Y-%m-%d'))
    contract_data["SignedDate"].append(signed_date.strftime('%Y-%m-%d'))
    
    if row["TransactionType"] == "Rent":
        end_date = start_date + timedelta(days=int(row["LeaseYear"] * 365))
    else:
        end_date = np.nan  # No end date for purchase contracts
    
    contract_data["EndDate"].append(end_date if pd.isna(end_date) else end_date.strftime('%Y-%m-%d'))
    
    # Update ContractStatus based on EndDate
    if row["TransactionType"] == "Purchase":
        contract_data["ContractStatus"].append("Active")
    else:
        if end_date < datetime(2024, 8, 1):
            contract_data["ContractStatus"].append(random.choice(["Terminated", "Expired"]))
        else:
            contract_data["ContractStatus"].append("Active")

# Create the DataFrame
contract_df = pd.DataFrame(contract_data)

# Set RenewalOption to None for purchase transactions
contract_df.loc[contract_df['ContractType'] == 'Purchase', 'RenewalOption'] = np.nan

# Remove TerminationClause for purchase contracts
contract_df.loc[contract_df['ContractType'] == 'Purchase', 'TerminationClause'] = np.nan

# Update SpecialTerms for purchase contracts with realistic terms
special_terms_purchase = [
    "Includes all appliances",
    "No HOA fees for the first year",
    "Seller to cover closing costs",
    "Home warranty included",
    "Property sold as-is",
    "Immediate move-in available",
    "Seller financing available",
    "Includes furniture"
]
contract_df.loc[contract_df['ContractType'] == 'Purchase', 'SpecialTerms'] = np.random.choice(special_terms_purchase, size=contract_df[contract_df['ContractType'] == 'Purchase'].shape[0])

# Replace 0.0 values in MonthlyRent with None (NaN)
contract_df['MonthlyRent'] = contract_df['MonthlyRent'].replace(0.0, np.nan)

# Sort the dataframe by SignedDate
contract_df = contract_df.sort_values(by='SignedDate').reset_index(drop=True)

# Reassign ContractID based on the chronological order of SignedDate
contract_df['ContractID'] = [f"C{i:04d}" for i in range(1, len(contract_df) + 1)]

# Save the final enhanced Contract table
contract_df.to_csv('/mnt/data/Contract_Enhanced_Sorted_Final.csv', index=False)

